In [1]:
import cv2
import os

healthy_folder = r"C:\Users\Rishabh Pandey\Desktop\ML\New folder2\healthy"
unhealthy_folder = r"C:\Users\Rishabh Pandey\Desktop\ML\New folder2\unhealthy"
defected_folder = r"C:\Users\Rishabh Pandey\Desktop\ML\New folder2\defectedcrops"

# Create defected folder 
if not os.path.exists(defected_folder):
    os.makedirs(defected_folder)

# Load unhealthy images as grayscale
unhealthy_templates = []
for fname in os.listdir(unhealthy_folder):
    if fname.lower().endswith(('.jpg', '.jpeg')):
        temp = cv2.imread(os.path.join(unhealthy_folder, fname), 0)
        if temp is not None:
            unhealthy_templates.append(temp)

if not unhealthy_templates:
    print("No unhealthy templates found.")
    exit()


for img_name in os.listdir(healthy_folder):
    if not img_name.lower().endswith(('.jpg', '.jpeg')):
        continue

    img_path = os.path.join(healthy_folder, img_name)
    img = cv2.imread(img_path)

    if img is None:
        print(f"Skipping invalid image: {img_name}")
        continue

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detected = False

    for template in unhealthy_templates:
        h, w = template.shape
        result = cv2.matchTemplate(gray_img, template, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(result)

        if max_val > 0.6:  # adjust threshold if needed
            top_left = max_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(img, top_left, bottom_right, (0, 0, 255), 2)
            cv2.putText(img, "Diseased Detected", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            detected = True
            break

    if not detected:
        cv2.putText(img, "Healthy", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    else:
        # Save detected image to defectedcrops folder
        save_path = os.path.join(defected_folder, img_name)
        cv2.imwrite(save_path, img)

    
    cv2.imshow("Potato", img)
    key = cv2.waitKey(50)  # Show each image for 50ms
    if key & 0xFF == ord('q'):
        break

print("Slideshow finished. Press any key to close.")
cv2.waitKey(0)
cv2.destroyAllWindows()


Slideshow finished. Press any key to close.
